In [1]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [2]:
df1 = pd.read_parquet(data_path + 'data/test1.parquet')
df2 = pd.read_parquet(data_path + 'data/test4.parquet')
df3 = pd.read_parquet(data_path + 'data/test6.parquet')
df4 = pd.read_parquet(data_path + 'data/test8.parquet')
df5 = pd.read_parquet(data_path + 'data/test3.parquet')

In [3]:
df0 = df1.merge(df2, on=['기준년월','ID'], how='left')
df0 = df0.merge(df3, on=['기준년월','ID'], how='left')
df0 = df0.merge(df4, on=['기준년월','ID'], how='left')
df0 = df0.merge(df5, on=['기준년월','ID'], how='left')
del df1
del df2
del df3
del df4
del df5
gc.collect()

26

In [4]:
df0[df0.select_dtypes(include='int64').columns] = df0.select_dtypes(include='int64').astype('int32')
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Columns: 354 entries, 기준년월 to 이용금액대
dtypes: float64(28), int32(321), object(5)
memory usage: 885.8+ MB


In [5]:
df00 = df0.loc[-100000:,:]
df00.to_parquet(data_path + 'test00.parquet', index=False)

In [6]:
x = df00.drop(['ID','기준년월','대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M'], axis=1)
import pickle
with open(data_path + 'data/01_model.dat', 'rb') as fp:
    test_model = pickle.load(fp)
    test_le_target = pickle.load(fp)

In [7]:
r1 = test_model.predict(x)
r1

array([3, 4, 3, ..., 4, 2, 4])

In [8]:
r2 = test_le_target.inverse_transform(r1)
r2

array(['D', 'E', 'D', ..., 'E', 'C', 'E'], dtype=object)

In [9]:
sub = pd.read_csv(data_path + 'sample_submission.csv')
sub

,ID,Segment
0,TEST_00000,A
1,TEST_00001,A
2,TEST_00002,A
3,TEST_00003,A
4,TEST_00004,A
...,...,...
99995,TEST_99995,A
99996,TEST_99996,A
99997,TEST_99997,A
99998,TEST_99998,A


In [11]:
r3 = r2[-100000:]
len(r3)

100000

In [12]:
sub['Segment'] = r3

In [13]:
sub.to_csv(data_path + 'submission1.csv', index=False)

In [15]:
fi = pd.DataFrame({'columns':x.columns, 'importance':test_model.feature_importances_})
fi.sort_values(by='importance', ascending=False)

,columns,importance
341,정상입금원금_B5M,0.093194
261,할부금액_무이자_3M_R12M,0.087004
339,정상청구원금_B5M,0.040215
170,잔액_한도소진율,0.027691
70,포인트_포인트_월적립_R3M,0.020670
...,...,...
26,유효카드수_체크_가족,0.000000
313,이용건수_C페이_R6M,0.000000
31,이용가능카드수_체크_가족,0.000000
320,이용건수_C페이_B0M,0.000000


In [16]:
fi.loc[fi['importance'] < 0.01,:]

,columns,importance
0,남녀구분코드,0.007291
1,연령,0.002248
2,회원여부_이용가능,0.000560
3,회원여부_이용가능_CA,0.002067
4,회원여부_이용가능_카드론,0.002140
...,...,...
343,연속무실적개월수_기본_24M_카드,0.000766
344,연속유실적개월수_기본_24M_카드,0.003758
345,신청건수_ATM_CA_B0,0.000408
346,신청건수_ATM_CA_R6M,0.001484
